In [2]:
import sys
sys.path.append("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition")

from core import get_dataset, get_metrics, argument_init, get_lattice_dataset
from lib import subTrainer
from data.DatasetLoadingHelper import load_ctc2021, load_sighan
#from models.bart.modeling_bart_v2 import BartForConditionalGeneration
from models.bert.modeling_bert_v2 import BertForFlat


In [3]:
import os
import random
import time
import logging
import argparse
from dataclasses import dataclass, field
from typing import Optional,Dict, Union, Any, Tuple, List

import numpy as np
import datasets
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import transformers
from transformers import (
    BertConfig,
    DataCollatorForSeq2Seq,
    AutoConfig,
    AutoTokenizer,
    HfArgumentParser,
    TrainingArguments,
    Seq2SeqTrainingArguments,
    set_seed,
)
from transformers import Trainer, Seq2SeqTrainer
from transformers import TrainingArguments
from transformers import trainer_utils, training_args
from transformers.trainer_pt_utils import nested_detach
from transformers import BertForMaskedLM
from transformers.file_utils import PaddingStrategy
from transformers.modeling_utils import PreTrainedModel
from transformers.tokenization_utils_base import BatchEncoding, PreTrainedTokenizerBase
from transformers.training_args import TrainingArguments

from core import get_dataset, get_metrics, argument_init
from lib import subTrainer  
from data.DatasetLoadingHelper import load_ctc2021, load_sighan
#from models.bart.modeling_bart_v2 import BartForConditionalGeneration
from models.bert.modeling_bert_v2 import BertForFlat

In [4]:
tokenizer_model_name_path="hfl/chinese-roberta-wwm-ext"

tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_model_name_path)


datasets, vocabs, embedding = get_lattice_dataset("sighan", path_head="../") 


Loading Dataset !
Read cache from cache/sighan_lattice_test.


In [5]:
#max_seq_len = max(*map(lambda x: max(x['seq_len']), datasets.values()))
#max_lattice_len = 640
config = BertConfig.from_pretrained(
    "hfl/chinese-roberta-wwm-ext"
)
config.num_labels = len(vocabs['label'])
config.vocab_size = len(vocabs['lattice'])
config.max_seq_len = 128#max_seq_len
config.max_lattice_len = 128#max_lattice_len
config.bert_hidden_size = config.hidden_size
config.hidden_size = 128#args.hidden_size
config.num_attention_heads = 8
config.rel_pos_init = 1
config.learnable_position = False#args.learnable_position
config.num_hidden_layers = 1#args.num_hidden_layers
config.position_embedding_path = None#args.position_embedding_path
config.use_crf = False#args.use_crf
config.position_type = "flat"#args.position_type
config.position_embedding = ""#args.position_embedding
config.position_fusion = "four"#args.position_fusion
config.position_first_layer = False#args.position_first_layer

model = BertForFlat(config=config, word_embedding=embedding)

In [6]:

test_data = datasets["train"][0]

In [10]:
print(test_data["pos_s"])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 0]


In [7]:
print(len(test_data["target"]))
print(len(test_data["input_ids"]))

72
73


In [8]:
import torch

test_data = datasets["train"][0]

def f(x):
    return torch.tensor([x])

model(
    f(test_data["input_ids"]),
    f(test_data["atten_masks"]),
    f(test_data["pos_s"]),
    f(test_data["pos_e"]), 
    f(test_data["target"]), 
)




torch.Size([1, 73, 5286])


{'loss': tensor(8.6147, grad_fn=<NllLossBackward>)}